# Agents Intensive Capstone Project

## Track: *Concierge Agents*

## Problem: *The Hyper-Local AI Culinary Logistics Agent*

Cooking at home is often hindered not by a lack of cooking skills, but by the friction of logistics. While the internet provides infinite recipe inspiration, the process of translating a digital recipe into physical ingredients located in specific, real-world stores is manually intensive.

Currently, a user must:
1. Find a recipe.
2. Manually cross-reference ingredients with their pantry.
3. Estimate quantities (converting "3 carrots" to purchasable weight).
4. Decide which local store carries the specific items (e.g., specialty spices vs. generic dairy).
5. Create a disorganized list that doesn't match the store layout.

## Objective

Develop an AI Agent that acts as an orchestration layer between user, Recipe APIs (by Google Search Tool) and Grocery/Map APIs (by Google Search Tool).

The Agent based on user's natural language request have to cravl foe a reciple, and provide 
products list for cooking specified dish. The list should be grouped by shops and contain also estimates of prices.

## Included feautres

- **Multi-agent system** - SequentialAgent, Agent powered by an LLM
- **Tools** - Google Search Tool
- **Observability** - Logging

## Implementation


### Importing secrets

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


### Import ADK components

In [2]:
from google.adk.agents import Agent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.adk.plugins.logging_plugin import LoggingPlugin
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


### Configure Retry Options

In [3]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

### Recipe resolver agent

In [4]:
recipe_resolver_agent = Agent(
    name="recipe_resolver",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that can answer resolve recipes.",
    instruction="You are a helpful assistant cooking assistant. Use Google Search for resolving recipes.",
    tools=[google_search],
)

print("✅ Recipe Resolver Agent defined.")

✅ Recipe Resolver Agent defined.


### Shopping list agent

In [5]:
shopping_list_agent = Agent(
    name="shopping_list",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that can order order cooking ingredients",
    instruction="Provide list cooking ingredients, group them by shop and provide approximate prices.",
    tools=[google_search],
)

print("✅ Shopping List Agent defined.")

✅ Shopping List Agent defined.


### Cooking Assistant Agent

Architecture
![](http://)

In [6]:
root_agent = SequentialAgent(
    name="CookingAssistant",
    sub_agents=[recipe_resolver_agent, shopping_list_agent],
)
print("✅ Cooking Assistant Agent defined.")

✅ Cooking Assistant Agent defined.


### Runner

In [7]:
runner = InMemoryRunner(
    agent=root_agent,
    plugins=[
        LoggingPlugin()
    ]
)

print("✅ Runner created.")

✅ Runner created.


### Example 

In [8]:
response = await runner.run_debug(
    ["I want to prepare hamburgers for dinner"]
)


 ### Created new session: debug_session_id

User > I want to prepare hamburgers for dinner
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-22044d2d-0fa7-4579-b821-f2eb0c19908a
[logging_plugin]    Session ID: debug_session_id
[logging_plugin]    User ID: debug_user_id
[logging_plugin]    App Name: InMemoryRunner
[logging_plugin]    Root Agent: CookingAssistant
[logging_plugin]    User Content: text: 'I want to prepare hamburgers for dinner'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-22044d2d-0fa7-4579-b821-f2eb0c19908a
[logging_plugin]    Starting Agent: CookingAssistant
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: CookingAssistant
[logging_plugin]    Invocation ID: e-22044d2d-0fa7-4579-b821-f2eb0c19908a
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: recipe_resolver
[logging_plugin]    Invocation ID: e-22044d2d-0fa7-4579-b821-f2eb0c19908a
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin